In [ ]:
# Code written by Anna Barwig
# partially automates statement generation
# this files makes use of the CheckList Libary: https://github.com/marcotcr/checklist @Ribeiro et al, 2020

!pip install datasets
!pip install checklist
import datasets
import checklist
from checklist.editor import Editor
import random
import re
from google.colab import drive
from pandas import read_excel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 27.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 43.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached qtconsole-5.5.1-py3-none-any.whl (123 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# starter script by https://gist.github.com/leonweber/43fde09bb27797b7e1da59604250c085
annotations = datasets.load_dataset("bigbio/sem_eval_2024_task_2", name="sem_eval_2024_task_2_source")
raw_texts = datasets.load_dataset("bigbio/sem_eval_2024_task_2", name="sem_eval_2024_task_2_ct")["train"]
id2record = {}
for instance in raw_texts:
  id2record[instance['clinical_trial_id']] = instance

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for bigbio/sem_eval_2024_task_2 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/sem_eval_2024_task_2
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release o

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating practice_test split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for bigbio/sem_eval_2024_task_2 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/sem_eval_2024_task_2
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
drive.mount('/content/drive')

def read_txt_file(path2file):
  with open(path2file) as input:
    return input.read().split("\n")

# reads SPECIALIST Lexicon and creates abbreviation-to-word and word-to-abbreviation dict
# LEXICON can be sourced from: https://uts.nlm.nih.gov/uts/umls/home
all_entries = {}
new_entry = {"entry":0}

for line in read_txt_file("/content/drive/MyDrive/LEXICON.txt"):
  match_found = re.search("[^{](.*=.*)", line.strip())
  if match_found:
    key, item = match_found.group().split("=")

    if key == "base":
      if new_entry["entry"]: all_entries[new_entry["entry"]] = new_entry
      new_entry = {"backronym":[], "spelling_variant":[]}
      new_entry[key] = item

    elif key == "acronym_of" or key == 'abbreviation_of':
      if "|" in item:
        backronym, entry_id = item.split("|")
      else:
         backronym, entry_id = item, ""
      new_entry["backronym"].append(backronym)

    elif key == "spelling_variant":
      new_entry[key].append(item)

    else: new_entry[key] = item

abbr2word = {all_entries[entry]["base"]:all_entries[entry]["backronym"] for entry in all_entries
             if len(all_entries[entry]["backronym"]) > 0}
word2abbr = {word:abbr for abbr, words in abbr2word.items() for word in words}

Mounted at /content/drive


In [ ]:
abbr2word["PBR"]

['peripheral benzodiazepine receptor',
 'peripheral-type benzodiazepine receptor',
 'peptide binding region',
 'parabrachial region',
 'packed bed reactor',
 'positive biopsy rate',
 'polybasic region']

In [ ]:
import random

def get_random_subsection(eligiblity_section, n=5):
  filtered = list(subsection.strip() for subsection in eligiblity_section
                              if 'inclusion criteria' not in subsection.lower()
                                  and 'exclusion criteria' not in subsection.lower()
                                  and 'disease characteristics' not in subsection.lower())
  return [filtered[random.randint(0, len(filtered) - 1)].strip() for _ in range(n)]

def find_comparisons(eligiblity_section):
  """
  searches for comparisons based on the pattern (medical term) (comparison type).
  medical terms are defined to be 1 - 2 window-sizes away from comparison type
  """
  comparisons = []
  for subsection in eligiblity_section:
    matches = re.findall(".+?(?=[=<>]+)|.+?(?=must be)|.+?(?=is less)|.+?(?=is more)|.+?(?=is equal)|.+?(?=is greater)", subsection)

    for found_pattern in matches:
      if found_pattern:

      # removes empty strings and selects window size of 1-2
        pattern_window = list(filter(bool, found_pattern.split(" ")))[-2:]
        word2abbr_lowercased = {word.lower():abbr for word, abbr in word2abbr.items()}

      # remove brackets
        brackets = "[\[()\]]"
        bigram = re.sub(brackets, "", " ".join(pattern_window))
        unigram = re.sub(brackets, "", " ".join(pattern_window[-1:]))

      # removes slashes from brigrams that appear to be unigrams
      # greedily selects first term
        if re.search("\/", unigram):
          unigram = re.split("\/", unigram)[0]

      # searches related context
        bigram_context = re.split(bigram, subsection).pop()
        unigram_context = re.split(unigram, subsection).pop()

      # removes beginning trails from context
        trail = '^([\(\)\[\]]+)'
        bigram_context = re.sub(trail, '', bigram_context)
        unigram_context = re.sub(trail, '', unigram_context)

      # check for word
        if bigram.lower() in word2abbr_lowercased:
          comparisons.append((bigram, bigram_context))
        elif unigram.lower() in word2abbr_lowercased:
          comparisons.append((unigram, unigram_context))
      # check for abbreviation -- note: works only with correct spelling
        elif bigram in abbr2word:
          comparisons.append((bigram, bigram_context))
        elif unigram in abbr2word:
          comparisons.append((unigram, unigram_context))

  return comparisons


def eligibility_based(primary_section, secondary_section=None):
  """Logical statements that are eligibility-based. These usually contain long sub-statements.
  It deals with the subproblem of what is eligible and being excluded from the trial.
  Statements might be re-phrased from the actual evidence provided
  or may contain no evidence present at all to increase the difficulty level for the model"""

  primary_comparisons = find_comparisons(primary_section)

  # sanity check: if comparison not found, add random subsections
  if not len(primary_comparisons):
    primary_comparisons = get_random_subsection(primary_section)

  if secondary_section is not None:
    secondary_comparisons = find_comparisons(secondary_section)
      # sanity check: if comparison not found, add random subsections
    if not len(secondary_comparisons):
       secondary_comparisons = get_random_subsection(secondary_section)
    primary_comparisons.extend(secondary_comparisons)

  # deal with sparse data problem
  if len(primary_comparisons) < 5:
    n_missing_data = 5 - len(primary_comparisons)
    primary_comparisons.extend(get_random_subsection(primary_section, n_missing_data))


  template = {"single":["A person with {PRIMARY_COMPARISON1},  {PRIMARY_COMPARISON2}, and {PRIMARY_COMPARISON3} are not eligilbe for the primary trial.", # contains spelling errors
                        "Participants of the primary trial {PRIMARY_COMPARISON1}, {PRIMARY_COMPARISON2}, {PRIMARY_COMPARISON3} but cannot {PRIMARY_COMPARISON4}.", # contains negative conjunction
                        "Men with {PRIMARY_COMPARISON} are not excluded from the primary trial.",
                        "Participants of the primary trial cannot be currently {PRIMARY_COMPARISON1}, {PRIMARY_COMPARISON2}, {PRIMARY_COMPARISON3} or {PRIMARY_COMPARISON4}",
                        "Eligible for the primary trial are female or male with {PRIMARY_COMPARISON1} and {PRIMARY_COMPARISON2}"], # perturbation
             "comparison":["The following conditions are required to be eligible for the secondary trial but not the primary trial; {PRIMARY_COMPARISON1}, {PRIMARY_COMPARISON2} and {PRIMARY_COMPARISON3}.", #contains negative conjunction
                            "Women with and {PRIMARY_COMPARISON1}, with {PRIMARY_COMPARISON2}, {PRIMARY_COMPARISON3} and {PRIMARY_COMPARISON4} are eligible for both the primary trial and the secondary trial",
                           "Patients with {PRIMARY_COMPARISON1}, who's {PRIMARY_COMPARISON2} are excluded from the secondary trial, but are eligible for the primary trial.",
                           "Patients diagnosed with {PRIMARY_COMPARISON} are eligible for the primary trial but not eligible for the secondary trial.",
                           "Neither {PRIMARY_COMPARISON} nor {PRIMARY_COMPARISON2} are required for participation in the primary trial and the secondary trial"]} # new sentence composition

  editor = Editor()
  primary_comparisons = list(map("".join, primary_comparisons))
  editor.add_lexicon("PRIMARY_COMPARISON", primary_comparisons, overwrite=True)

  if secondary_section is not None:
    x = template["comparison"][random.randint(0, len(template["comparison"])-1)]
    ret = editor.template(x, remove_duplicates=True)
  else:
    y = template["single"][random.randint(0, len(template["single"])-1)]
    ret = editor.template(y, remove_duplicates=True)

  if ret.data:
    return ret.data[0]

def detect_drugs(intervention_section, n=2):
  drugs = []
  filtered = list(filter(lambda section_list: "INTERVENTION 1:" not in section_list and "INTERVENTION 2:" not in section_list, intervention_section))
  for subsection in filtered:
    drug_found = re.search(".+?(?=:)", subsection)
    if drug_found: drugs.append(drug_found.group(0))
  if not len(drugs): return [filtered[random.randint(0, len(filtered) - 1)].strip() for _ in range(n)]
  return drugs

def detect_events(adverse_events_section):
   filtered = list(filter(lambda section_list: "Adverse Events" not in section_list and "Total" not in section_list, adverse_events_section))
   return [re.search("[a-zA-Z\\/\s]+[a-zA-Z\\/\s]*", event).group().strip() for event in filtered]

def detect_measurement(results_section, search_abbreviation=True):
  #if search_abbreviation:
   # matchFound = re.search("(?<=\().+?(?=\))", results_section[1])
  #  if matchFound: return [matchFound.group().strip()]
  return [results_section[1].strip().lower()]


def intervention_based(primary_section, secondary_section=None):
  """Logical statements that are intervention-based.
  These usually contain very generic statements that are heavily section-dependent"""

   # what kind of treatment
   # whether it is a placebo or test group
   # if interventions are explained
   # details of treatment: dosage, type, duration

  template = {"single":["no placebo treatment is used in cohort 2 of the primary trial.",
                        "the primary trial is testing a [treatment].",
                        "in the primary trial the interventions are explained in the intervention section.",
                        "The intervention section for the primary trial only details the dosage and type but not the duration of the intervention."],
              "comparison":[
                  "Placebo treatment is used in cohort 1 of the primary trial, but there is only a test group in the secondary trial.",
                  "None of the subjects in the primary trial are required to injest any pills, whereas both cohorts of the secondary trial must take a weekly tablet.",
                  "the secondary trial investigates a [treatment]. but not the primary trial."]}

  if secondary_section is not None:
    return template["comparison"][random.randint(0, len(template["comparison"])-1)]
  else: return template["single"][random.randint(0, len(template["single"])-1)]


def results_based(primary_section, secondary_section=None):
  editor = Editor()
  primary_measurement = detect_measurement(primary_section, False)

  if secondary_section is not None:
    secondary_measurement = detect_measurement(secondary_section, False)
    primary_measurement.extend(secondary_measurement)

  editor.add_lexicon("MEASUREMENT", primary_measurement, overwrite=True)


  template = {"single":["the primary trial either use overall response rate, tumour response rate or progression-free survival rate as its outcome measurement.",
                        "the primary trial does not measure {MEASUREMENT}"],
              "comparison":["the secondary trial and the primary trial do not use similar outcome measures.",
                            "Unlike the primary trial, the secondary trial is measuring {MEASUREMENT}."]}

  if secondary_section is not None:
     ret = editor.template(template["comparison"][random.randint(0, len(template["comparison"])-1)])
  else:
     ret = editor.template(template["single"][random.randint(0, len(template["single"])-1)])

  if ret.data:
    return ret.data[0]


def abbreviation(section_type, primary_section, secondary_section=None):
  """aims to generate statements where either the premise contains a spelled out variant of the abbreviation used in the statement
  or the other way around, e.g. "twice a day" in premise, "BID" in statement; does not apply to eligibility section"""
  editor = Editor()

  if section_type == "intervention":
    primary_drugs = detect_drugs(primary_section)
    if secondary_section is not None:
      secondary_drugs = detect_drugs(primary_section)
      primary_drugs.extend(secondary_drugs)
    if not len(primary_drugs): return
    editor.add_lexicon("DRUG", primary_drugs, overwrite=True)

    template = {"single":["the primary trial subjects recieving {DRUG} are administered it topically [twice/three times] daily for approximately [duration]."], # tests for BID or TID
      "comparison": ["In the primary trial and the secondary trial the only drugs administered orally are {DRUG1} and {DRUG2}.",                                 # tests for PO tag
                     "participants of cohort 1 in the primary trial and all participants of the secondary trial take [NUM] milligram of {DRUG} PO QD."]}

  elif section_type == "adverse_events":
    primary_events = detect_events(primary_section)
    primary_abbr = list(filter(bool, [word2abbr.get(word.lower(), "") for word in primary_events]))
    if secondary_section is not None:
      secondary_events = detect_events(secondary_section)
      secondary_abbr = list(filter(bool, [word2abbr.get(word.lower(), "") for word in secondary_events]))
      primary_abbr.extend(secondary_abbr)

    if not len(primary_abbr): return
    editor.add_lexicon("ABBR_EVENT", primary_abbr, overwrite=True)

    template = {"single":["Most of the cases of {ABBR_EVENT} in the primary trial, were in cohort 2.",
                          "None of the adverse events which occurred in the primary trial were {ABBR_EVENT} related."],
                "comparison":["the secondary trial records several {ABBR_EVENT} adverse events, whereas the primary trial doesn‚Äö√Ñ√¥t.",
                              "Both the primary trial and the secondary trial had no cases of {ABBR_EVENT}"]}

  elif section_type == "results":
    # tests outcome measurements as abbreviations
    # uses set abbreviations as default values

   # primary_measurement = detect_measurement(primary_section, False)
   # if secondary_section is not None:
   #   secondary_measurement = detect_measurement(secondary_section, False)
   #   primary_measurement.extend(secondary_measurement)

   # print(primary_measurement)
  #  editor.add_lexicon("MEASUREMENT", primary_measurement, overwrite=True)

    template = {"single":["The primary trial studies PBR",
                          "The primary trial studies PFS",
                          "The primary trial studies DLTs"],
                "comparison":["The primary trial and the secondary trial are not studying PFS, PBR or DLTs.",
                              "The primary trial and the secondary trial are not studying PFS or DLTs.",
                              "The primary trial and the secondary trial are not studying PBR or PFS."]}

  #  template = {"single":["the primary trial does not study {MEASUREMENT}",
   #                       "the primary trial studies {MEASUREMENT}"],
    #            "comparison":["the primary trial and the secondary trial are not studying PFS, PBR or DLTs.",
    #                          "While the secondary trial studies {MEASUREMENT1}, the primary trial studies {MEASUREMENT2}",
    #                          "the primary trial and the secondary trial measure the {MEASUREMENT}"]}

  if secondary_section is not None:
     ret = editor.template(template["comparison"][random.randint(0, len(template["comparison"])-1)], remove_duplicates=True)
  else:
     ret = editor.template(template["single"][random.randint(0, len(template["single"])-1)], remove_duplicates=True)

  if ret.data:
    return ret.data[0]

def retrieve_section_content(section_id, primary_trial, secondary_trial):
    primary_section = id2record[primary_trial][section_id]
    if str(secondary_trial) != "nan":
      secondary_section = id2record[secondary_trial][section_id]
      return primary_section, secondary_section
    else: return primary_section, None


In [ ]:
import json


def filter_category(new_section):
 return df.index[df["Problem Category"] == new_section].tolist()

df = read_excel('/content/drive/MyDrive/selected_instances_merged.xlsx')
category = ("Eligibility based", "Intervention based", "Results based", "Abbreviation")
# # Credit: https://stackoverflow.com/a/1993840) @sykora
category2indices = dict(zip(category, map(filter_category, category)))

idx2new_statement = {}
for category, indices in category2indices.items():
  for idx in indices:
    section_type = df["New section type"][idx].lower()
    # adjust spelling mismatches
    if section_type == "adverse events": section_type = "adverse_events"
    primary_content, secondary_content = retrieve_section_content(section_type, df["primary_id"][idx], df["secondary_id"][idx])

    #if category == "Eligibility based":
    #  new_statement = eligibility_based(primary_content, secondary_content)
    #  idx2new_statement[idx] = new_statement
   # if category == "Intervention based":
    #  new_statement = intervention_based(primary_content, secondary_content)
    #  idx2new_statement[idx] = new_statement
    #if category == "Results based":
    #  new_statement = results_based(primary_content, secondary_content)
     # idx2new_statement[idx] = new_statement
    if category == "Abbreviation":
      new_statement = abbreviation(section_type, primary_content, secondary_content)
      idx2new_statement[idx] = new_statement





In [ ]:
with open('/content/drive/MyDrive/abbreviation_statements_draft.json', 'w', encoding='utf-8') as f:
    json.dump(idx2new_statement, f, indent=3)


In [ ]:
abbr2word["PNE"]

['primary nocturnal enuresis',
 'percutaneous nerve evaluation',
 'peripheral nerve evaluation',
 'plasma norepinephrine',
 'plasma norepinephrine concentration',
 'prenatal nicotine exposure',
 'pudendal nerve entrapment',
 'pneumonia']